In [1]:
import docx2txt
from book_builder.book_builder import BookBuilder

In [2]:
import sqlite3
conn = sqlite3.connect('data/kcbible.db')
db = conn.cursor()

In [28]:
book_txt = docx2txt.process('docx_normalized/Mate.docx')
lines = [line.strip() for line in book_txt.split('\n\n') if line.strip() != '']
bb = BookBuilder(lines[0])
for line in lines[1:]:
    bb.process(line)
book = bb.build()
print(f"book {book.title} has {len(book.chapters)} chapters with a combined total of {sum([len(c.verses) for c in book.chapters])} verses")

book MATE THUTHANGNI has 28 chapters with a combined total of 1068 verses


In [29]:
db.execute('insert into book(name, language) values(?,?)',('Mate','Kaang Chin'))
book_id = db.lastrowid
conn.commit()

In [30]:
insert_chapter = """
    insert into chapter(number, title, book_id)
    values(?,?,?)
    """

In [31]:
insert_verse = """
    insert into verse(number, title, text, chapter_id)
    values(?,?,?,?)
"""

In [32]:
for c in book.chapters:
    chapter_params = (c.number, c.title if c.title else None, book_id)
    db.execute(insert_chapter, chapter_params)
    chapter_id = db.lastrowid
    verse_params = [(v.number,v.title if v.title else None,v.text,chapter_id) for v in c.verses]
    db.executemany(insert_verse, verse_params)
conn.commit()

In [35]:
def test(c,v):
    return c.number == 22

def getVerses(b):
    return [f"{c.number}:{v.number} {v.text}" for c in b.chapters for v in c.verses if test(c,v)]

getVerses(book)

['22:1 Jesu ein nuetnak awn thu a toekbai hei.',
 '22:2 Fanram kha, a ca ei chruumnak aw ai biloupueng ei Hrangpuei awn toeng kue.',
 '22:3 Chruum buh ai la, a khueu hei khoen, oeh lawhluepuengtu khueu bai yeila, a saiitu tueih hei kue.',
 '22:4 Vantangkong awn boivok athauhetu, ka ngon ein avan ka ripkhoen kue, chruumnak la, tuek hei khoentu law u eila, vawtoek hei u tikue, saii feitu tueih hei bai kue.',
 '22:5 Khanaphi oeh ngai u ein am lou la ceit hei u ein afeitu kha sumlei la ceit u kue.',
 '22:6 Apoih ein a saiitu pang ein thekhana khap u ein ngon hei u kue.',
 '22:7 Hrangpuei ein a yaakah, a kawtheh ein, am khawtu mei awn khum u kom ngon hei u eila, a raiikaptu tuei hei kue.',
 '22:8 Khakonna a saiitu a toek hei, chruumnak kha atangte la krakue, khanaphi ka khueutu oeh hawina u kue a ti.',
 '22:9 Khatiein lampuei la ceit u, nam hmuh poeng chruum buh ai la khueu hei u a ti.',
 '22:10 Asaiitu kha lampoengah ceit u ein am hmuh avan chrangni chrangthe atoengla, am bumnak awn chruum

In [10]:
conn.close()